In [50]:
import os
from dotenv import load_dotenv

from geopy.geocoders import GoogleV3

import json

In [4]:
# Opening JSON file
f = open('all_cl_2.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [6]:
len(data)

2241

## update json key name

In [7]:
def getloc_search(present, limit):
    if present == "None" or present == "":
        return limit
    else:
        return present + ' ' + limit

In [21]:
def getid(year):
    
    if year in id_array.keys():
        id_array[year] = id_array[year] + 1;
        return year + str(id_array[year]);
    else:
        id_array[year] = 1;
        return year + str(id_array[year]);


In [26]:
def valid_reason(d):
    if "reason" in d.keys():
        return d["reason"]
    else:
        return ""

In [27]:
data_updateName = []
id_array = {"2000": 0}

for d in data:
    obj = {
        "id": getid(d["year"]),
        "coname": d["newname"],
        "loc_search": getloc_search(d["presentname"],d["limits"]),
        "reason": valid_reason(d),
        "year": d["year"]
    }
    data_updateName.append(obj)
    print(obj)

{'id': '20031', 'coname': 'Steven Bells Belson Beach Way', 'loc_search': 'Beach 92nd Street from Rockaway Beach Boulevard to the boardwalk', 'reason': 'Borough of Queens', 'year': '2003'}
{'id': '20032', 'coname': 'Danny Pesce Place', 'loc_search': 'Scranton Avenue between Pompey Avenue and Wainwright Avenue', 'reason': 'Borough of Staten Island', 'year': '2003'}
{'id': '20033', 'coname': 'Firefighter Christopher Mozzillo Place', 'loc_search': 'Amber Street between Amboy Road and Shadow Lane', 'reason': 'Borough of Staten Island', 'year': '2003'}
{'id': '20034', 'coname': 'Adriana Scibetta Street', 'loc_search': 'Elmbank Street between Arden Avenue and Harold Avenue', 'reason': 'Borough of Staten Island', 'year': '2003'}
{'id': '20035', 'coname': 'Kenneth W. White Street', 'loc_search': 'Weiner Street between Richmond Valley Road and Bascomb Avenue', 'reason': 'Borough of Staten Island', 'year': '2003'}
{'id': '20036', 'coname': 'Laurence Curia Place', 'loc_search': 'Odell Place betwee

{'id': '2017138', 'coname': 'Amanda Clarke Way', 'loc_search': 'At the intersection of 137th Avenue and 227th Street', 'reason': 'Amanda Leah Clarke was a community activist who resided in Queens. She had a deep passion for civic, community and political engagement and was actively engaged in several community groups such as the Federated Blocks of Laurelton, where she served as a block watcher during the crime spikes of the 1970’s and 80’s, and the Laurelton Garden Club.  After the murder of her son Michael in 2002, she teamed up with friends and community members to form the Michael Arthur Clarke Foundation which provides scholarships to deserving students in southeast Queens.  In addition to her community activism, she was elected to be the Democratic Party District Leader and in 2001 and she ran for the New York City Council in the 31st District of Queens. Although she did not win, she remained actively involved in numerous campaigns under the titles of volunteer, donor, and fundra

In [28]:
len(data_updateName)

2241

In [29]:
with open("raw_to_GIS_crowdsource.json", "w") as outfile:
    json.dump(data_updateName, outfile)

## GEO Coding

In [33]:
data_updateName[9]

{'id': '200310',
 'coname': 'Edward K. “Eddie” Oliver Place',
 'loc_search': 'Florence Place between Seguine Avenue and Elizabeth Place',
 'reason': 'Borough of Staten Island',
 'year': '2003'}

### Google maps services

In [89]:
import googlemaps
from time import sleep

In [93]:
load_dotenv()
MYAPI = os.getenv('MYAPI')

gmaps = googlemaps.Client(key=MYAPI)

In [94]:
# Geocoding an address
geocode_result = gmaps.geocode('Florence Place between Seguine Avenue and Elizabeth Place')


In [55]:
print(geocode_result)

[{'address_components': [{'long_name': 'Seguine Avenue', 'short_name': 'Seguine Ave', 'types': ['route']}, {'long_name': "Prince's Bay", 'short_name': "Prince's Bay", 'types': ['neighborhood', 'political']}, {'long_name': 'Staten Island', 'short_name': 'Staten Island', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Richmond County', 'short_name': 'Richmond County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'New York', 'short_name': 'NY', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '10309', 'short_name': '10309', 'types': ['postal_code']}], 'formatted_address': 'Seguine Ave, Staten Island, NY 10309, USA', 'geometry': {'bounds': {'northeast': {'lat': 40.5265873, 'lng': -74.19592039999999}, 'southwest': {'lat': 40.5115473, 'lng': -74.2017906}}, 'location': {'lat': 40.5191984, 'lng': -74.1971456}, 'location_type': 'GEOME

In [84]:
def format_geo_result(result, obj):
    ### valid check??
    
    try:

        loc = result[0]["formatted_address"]
        borough = get_borough(result[0]["address_components"])
        postal = get_postal(result[0]["address_components"])
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        long = geocode_result[0]["geometry"]["location"]["lng"]

        newobj = {'id': obj["id"],
             'coname': obj["coname"],
             'location': obj["loc_search"],
             "lat": lat,
             "long": long,
             "borough": borough,
             "loc_result":loc ,
             "postal": postal,
             'reason': obj["reason"],
             'year': obj["year"]}
        return newobj
    
    except:
        
        print(obj["id"])
        return obj


In [56]:
def get_postal(components):
    for item in components:
        if "postal_code" in item["types"]:
            return item["long_name"]

In [57]:
def get_borough(components):
    for item in components:
        if "sublocality" in item["types"]:
            return item["long_name"]

In [86]:
format_geo_result("",data_updateName[9] )

200310


{'id': '200310',
 'coname': 'Edward K. “Eddie” Oliver Place',
 'loc_search': 'Florence Place between Seguine Avenue and Elizabeth Place',
 'reason': 'Borough of Staten Island',
 'year': '2003'}

In [71]:
geocode_result[0]["geometry"]["location"]

{'lat': 40.5191984, 'lng': -74.1971456}

### calling API

In [91]:
data_updateName[1000]

{'id': '200914',
 'coname': 'Jose Sucuzhañay Place',
 'loc_search': 'At the intersection of Bushwick Avenue and Kossuth Place',
 'reason': 'Reverend Dr. Jerry Mosby was the Pastor for the    in the  and a very active member of the community.  He also served as the Community Chaplain for the 43rd Precinct and was a retired Pastor for the Bronx Network Health and Hospital Corporation.  He was also vital in organizing Hurricane Relief Drives in the community, most memorable being Katrina.',
 'year': '2009'}

In [95]:
data_geo = []

In [96]:
for obj in data_updateName:
    geocode_result = gmaps.geocode(obj["loc_search"])
    sleep(0.1)
    data_geo.append( format_geo_result(geocode_result, obj ) )

20037
20038
20039
200315
200316
200318
200321
200322
200325
200331
200344
200371
200372
200374
200379
200384
200390
200396
2003104
2003105
2003106
2003112
2003113
2003115
2003118
2003119
2003122
2003128
2003141
2003168
2003169
2003171
2003176
2003181
2003187
2003193
2003208
2003209
2003212
2003221
2003235
2003237
2003240
2003243
2003245
2003248
2003249
2003250
2003251
2003256
2003260
2003263
2003264
2003265
2003266
2003267
2003268
2003269
2003273
2003274
2003276
2003279
2003284
2003291
2003294
2003313
2003317
2003318
2003321
2003323
2003325
2003328
2003332
2003339
2003343
2003345
2003346
2003347
2003348
2003350
2003355
2003356
2003357
2003358
2003359
2003367
2003369
2003370
2003373
2003375
2003378
2003380
2003381
2003382
2003384
2003385
20043
20045
200411
200413
200416
200417
200420
200425
200427
200436
200449
200458
200470
200471
200477
200478
200498
2004110
2004113
2004120
2004134
2004136
2004150
2004165
2004175
20055
20056
200519
200522
200523
200524
200528
200529
200534
200537
2005

In [97]:
len(data_geo)

2241

In [98]:
with open("GIS_crowdsource.json", "w") as outfile:
    json.dump(data_geo, outfile)